# How to Establish an Authenticated Connection to the Coinbase Cloud Advanced Trade REST API

by Nils Ackermann

## Introduction

In case you want to get started as a crypto investor and you want to take matters into your own hands, it is important that you are able to connect to a currency exchange such as [coinbase.com](https://coinbase.com). This is the essential first step to start data analysis and automation and move into algorithmic trading. In this notebook you will learn the following:

- How to connect to the authorized coinbase.com REST API via Python
- How to load your personal acount and order data from Coinbase into Pandas

The Coinbase Cloud API can be found [here](https://docs.cloud.coinbase.com/exchange/reference/).

## Step 0 - Setup your Python Virtual Environment

- Create a virtual Python environment in the folder of this notebook:

````
python3 -m venv venv
````

- Activate the virtual environment:

````
source venv/bin/activate
````

- Install all necessary libraries:

````
pip install -r requirements.txt 
````

- Change to the Python Kernel in your IDE that you use to open the Jupyter Notebook like Visual Studio Code

## Step 1 - Create an API Key and API Secret for your Coinbase Account

Just follow the instructions on the Coinbase website on how to create an API Key and the API Secret.

**Remark:** Creating an API Key on coinbase.com might not work with Safari - try Chrome browser instead. Also, Coinbase delays the activation of the key by 48 hours.

**Attention:** You must keep this key private. It must not be shared with anyone!

Once you have created the key, please create a file in the same folder as this Jupyter Notebook with the following command:

````
touch ./.env
````

The file must hold the following environment variables. The file content should look something like this:

````
export CB-API-KEY=<your-coinbase-api-key>
export CB-API-SECRET=<your-coinbase-api-secret>
````

## Step 2 - Imports, Libraries and Custom Functions

We need the following main Python imports:

- **Requests:** For connecting to the REST endpoints from Coinbase, we will use [this](http://docs.python-requests.org/en/master/) great HTTP library. Very user friendly. Only a few lines of code to connect via REST. 
- **Pandas:** For data analysis and manipulation we use [Pandas](https://pandas.pydata.org).
- **Json:** We are dealing with json format, therefore let's import it. The documentation is [here](https://docs.python.org/3/library/json.html)
- **python-dotenv** This is a convenient library to access the environment variables that hold our secret API-Key

In [ ]:
import pandas as pd
import json
import hmac
import hashlib
import requests
from datetime import datetime, timedelta
import time
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from urllib.error import HTTPError
from dotenv import load_dotenv
import os

In [ ]:
# Load environment variable holding your secret key
load_dotenv()
CB_API_KEY = os.environ.get('CB-API-KEY')
CB_SECRET_KEY = os.environ.get('CB-API-SECRET')

In [ ]:
def cb_connect(url_path, limit=50, cursor=''):
    url_prefix = 'https://coinbase.com'
    url = url_prefix + url_path
    secret_key = CB_SECRET_KEY
    api_key = CB_API_KEY
    timestamp = str(int(time.time()))
    method = 'GET'
    body = ''
    message = timestamp + method + url_path.split('?')[0] + body 
    signature = hmac.new(secret_key.encode('utf-8'), message.encode('utf-8'), digestmod=hashlib.sha256).digest()
    headers = {'accept': 'application/json','CB-ACCESS-SIGN':signature.hex(), 'CB-ACCESS-KEY':api_key, 'CB-ACCESS-TIMESTAMP': timestamp}
    url=url+'?limit='+str(limit)
    if cursor!='':
        url=url+'&cursor='+cursor
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        print(f'HTTP connection {url} successful!')
        return response
    except HTTPError as http_err:
        print(f'HTTP error occurred: {http_err}')
    except Exception as err:
        print(f'Other error occurred: {err}')

def cb_get_all_accounts():
    has_next = True
    cursor = ''
    lst_accounts = []
    while has_next:
        response = cb_connect(url_path='/api/v3/brokerage/accounts',
            limit=50,
            cursor=cursor)
        json_accounts = json.loads(response.text)
        has_next = json_accounts['has_next']
        cursor = json_accounts['cursor']
        tmp_df_accounts = pd.json_normalize(json_accounts, record_path =['accounts'])
        tmp_lst_accounts = tmp_df_accounts.values.tolist() 
        lst_accounts.extend(tmp_lst_accounts)
    # Create dataframe from list at the end to improve performance
    df_accounts = pd.DataFrame(lst_accounts)
    # Add column names to final dataframe
    df_accounts.columns = tmp_df_accounts.columns.values.tolist()
    return df_accounts

def cb_get_all_orders():
    has_next = True
    cursor = ''
    lst_orders = []
    while has_next:
        response = cb_connect(url_path='/api/v3/brokerage/orders/historical/batch',
            limit=100,
            cursor=cursor)
        json_orders = json.loads(response.text)
        has_next = json_orders['has_next']
        cursor = json_orders['cursor']
        tmp_df_orders = pd.json_normalize(json_orders, record_path =['orders'])
        tmp_lst_orders = tmp_df_orders.values.tolist() 
        lst_orders.extend(tmp_lst_orders)
    # Create dataframe from list at the end to improve performance
    df_orders = pd.DataFrame(lst_orders)
    # Add column names to final dataframe
    df_orders.columns = tmp_df_orders.columns.values.tolist()
    return df_orders
    

## Step 3 - Account Data

We will use the above defined function to retrieve all account data. We will then filter the data to only display crypto currencies that are relevant for this coinbase user (meaning available balance or hold value is greater 0).

In [ ]:
# All float numbers to show comma seperators when printing:
pd.options.display.float_format = '{:,.2f}'.format

df_accounts = cb_get_all_accounts()
df_accounts.rename(columns={'available_balance.value': 'available_balance','hold.value': 'hold'}, inplace=True)
df_accounts['available_balance'] = df_accounts['available_balance'].astype(float)
df_accounts['hold'] = df_accounts['hold'].astype(float)
df_accounts = df_accounts.query('available_balance > 0 or hold > 0')
df_accounts.drop(['available_balance.currency','hold.currency','default','created_at','updated_at','deleted_at','ready'], axis=1, inplace=True)
df_accounts[['currency','available_balance','hold']]

## Step 4 - Order Data

Next, we will use the above defined function to retrieve all orders (independent of status).

In [ ]:
df_orders = cb_get_all_orders()
df_orders.rename(columns={'order_configuration.limit_limit_gtc.base_size': 'base_size','order_configuration.limit_limit_gtc.limit_price': 'limit_price','order_configuration.limit_limit_gtc.post_only': 'post_only'}, inplace=True)
df_orders['base_size'] = df_orders['base_size'].astype(float)
df_orders['limit_price'] = df_orders['limit_price'].astype(float)
df_orders.drop(['user_id'], axis=1, inplace=True)
df_orders[['product_id','side','status','order_type','base_size','limit_price']]